In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df_id = df.iloc[:, 0]
df_id.tail()

501946    501946
501947    501947
501948    501948
501949    501949
501950    501950
Name: id, dtype: int64

In [6]:
df_users = df.iloc[:, 7:-15]
df_users.tail()

,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3
501946,1,1,2,2,1192,935,1228,3,1,1,59,4,95
501947,1,6,2,1,118,113,110,4,1,1,105,142,95
501948,1,7,4,1,147,46,145,4,1,1,59,127,139
501949,1,1,2,1,46,147,145,4,1,1,251,49,258
501950,1,6,3,5,147,46,145,6,1,1,278,31,49


In [7]:
df_contents = df.iloc[:, -15:-2]
df_contents.tail()

,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn
501946,3,3,5,1,1,2,354,147,1,5,65,503156,285850
501947,3,3,10,2,1,2,163,120,1,4,142,676255,456996
501948,3,1,5,1,1,2,438,147,2,7,65,484528,293258
501949,3,2,5,1,1,2,660,147,3,4,259,456330,273797
501950,3,3,5,1,1,2,786,147,2,5,289,235596,176650


In [8]:
df_target = df.iloc[:, -1]
df_target.tail()

501946    1
501947    1
501948    1
501949    1
501950    1
Name: target, dtype: int64

In [35]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv

In [10]:
user_id = df_target.index.to_numpy().reshape(-1, 1)
target = df_target.to_numpy().reshape(-1, 1)

In [11]:
direct1 = torch.tensor(np.concatenate((user_id, target), axis=1), dtype=torch.long)
direct2 = torch.tensor(np.concatenate((target, user_id), axis=1), dtype=torch.long)

In [12]:
edge_index = torch.concat((direct1, direct2))

In [13]:
edge_attr = torch.tensor(df_contents.values, dtype=torch.long)

In [14]:
x = torch.tensor(df_users.values, dtype=torch.long)

In [15]:
y = torch.tensor(df_target.values.reshape(-1), dtype=torch.long)

In [16]:
data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y, edge_attr=edge_attr)

In [27]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(data.num_features, 16, dim=1, kernel_size=2)
        self.conv2 = SplineConv(16, 2, dim=1, kernel_size=2)

    def forward(self):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)
    
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    log_probs, accs = model(), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = log_probs[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)